In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
from transformers import pipeline

summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

2025-08-20 22:07:12.571501: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755727632.827077      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755727632.894878      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cuda:0


In [3]:
def summarize_text(summarizer, text, max_length = 180, min_length = 50, do_sample = False):
  summary = summarizer(text, max_length = max_length, min_length = min_length, do_sample = do_sample)
  return summary[0]['summary_text']

In [4]:
article = """What is Text Summarization?

Text summarization is the process of condensing a large text document into a shorter version while preserving its key information and meaning. The goal of text summarization is to extract the most important information from a text document and present it in a concise and comprehensible form. This can be done through techniques such as keyword extraction, sentence extraction, or abstractive summarization. Text summarization has various applications including news aggregation, content analysis, and information retrieval.
How text summarization performed using Transformers?

Text summarization using Transformers can be performed in two ways: extractive summarization and abstractive summarization.

    Extractive summarization: In this approach, the most important sentences or phrases from the original text are selected and combined to form a summary. This can be done using algorithms such as TextRank, which uses graph-based algorithms to rank sentences based on their relevance and importance. Transformers can be used to process the text, extract features, and perform sentence ranking.
    Abstractive summarization: In this approach, a new summary is generated by understanding the context of the original text and generating new phrases and sentences that summarize its content. This can be done using techniques such as encoder-decoder models, where the encoder processes the input text to extract its features and the decoder generates the summary. Transformers can be used as the encoder or decoder in this architecture.

In both extractive and abstractive summarization, Transformers can be trained on large amounts of text data to learn the patterns and relationships between words, sentences, and documents, making them well-suited for text summarization tasks."""
summarize_text(summarizer, article)

'Text summarization is the process of condensing a large text document into a shorter version while preserving its key information and meaning. Text summarization has various applications including news aggregation, content analysis, and information retrieval. Transformers can be used to process text, extract features, and perform sentence ranking.'

In [5]:
from transformers import pipeline

qa = pipeline("question-answering", model="distilbert/distilbert-base-cased-distilled-squad")

config.json:   0%|          | 0.00/473 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/261M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cuda:0


In [6]:
def answering_questions(qa_model, question, context):
  answer = qa(question = question, context = context)
  return answer['answer'], answer['score']

In [7]:
context_ex = """
Artificial Intelligence (AI) is a branch of computer science that aims to build machines
capable of performing tasks that typically require human intelligence.
These tasks include reasoning, learning from experience, making decisions,
understanding natural language, and recognizing patterns.

In healthcare, AI is used to analyze medical images, predict patient outcomes,
and even assist in surgeries with robotic systems.
For example, AI algorithms can detect early signs of diseases such as cancer
from radiology scans with accuracy comparable to expert doctors.

In finance, AI helps banks and institutions detect fraud,
analyze risks, and provide personalized recommendations to customers.
It is also widely used in stock market predictions and algorithmic trading.

In transportation, AI powers self-driving cars,
optimizes traffic management systems in smart cities,
and improves logistics by predicting the best delivery routes.

However, AI also raises ethical concerns such as job displacement,
privacy issues, and the possibility of bias in automated decision-making.
Governments and organizations are working on policies and regulations
to ensure that AI is developed responsibly and benefits society as a whole.
"""

In [8]:
answering_questions(qa,'What are some ethical concerns of AI?', context_ex)

('job displacement,\nprivacy issues', 0.10792221128940582)

In [9]:
answering_questions(qa, 'Which industry uses AI for fraud detection?', context_ex)

('banks and institutions', 0.23580767214298248)

In [10]:
import gradio as gr
from transformers import pipeline

# Load models safely
def load_models():
    summarizer = pipeline(
        "summarization",
        model="facebook/bart-large-cnn",
        tokenizer="facebook/bart-large-cnn"
    )
    qa = pipeline(
        "question-answering",
        model="distilbert-base-cased-distilled-squad",
        tokenizer="distilbert-base-cased-distilled-squad"
    )
    return summarizer, qa

summarizer, qa = load_models()

# Define functions
def summarize_text(text):
    if not text.strip():
        return "⚠️ Please enter some text."
    summary = summarizer(text, max_length=130, min_length=30, do_sample=False)
    return summary[0]['summary_text']

def answer_question(context, question):
    if not context.strip() or not question.strip():
        return "⚠️ Please provide both context and question."
    result = qa(question=question, context=context)
    return result['answer']

# Build Gradio Interface
with gr.Blocks() as demo:
    gr.Markdown("# 🤖 Text Summarizer & Question Answering App")
    gr.Markdown("Paste some text, then summarize it or ask a question about it!")

    with gr.Tab("📌 Summarization"):
        text_input = gr.Textbox(label="Enter Text", lines=10, placeholder="Paste your text here...")
        summarize_btn = gr.Button("Summarize")
        summary_output = gr.Textbox(label="Summary", lines=5)
        summarize_btn.click(summarize_text, inputs=text_input, outputs=summary_output)

    with gr.Tab("❓ Question Answering"):
        context_input = gr.Textbox(label="Context", lines=10, placeholder="Paste your text here...")
        question_input = gr.Textbox(label="Question", placeholder="Type your question here...")
        qa_btn = gr.Button("Get Answer")
        answer_output = gr.Textbox(label="Answer", lines=2)
        qa_btn.click(answer_question, inputs=[context_input, question_input], outputs=answer_output)

# Launch app
if __name__ == "__main__":
    demo.launch()


Device set to use cuda:0


config.json:   0%|          | 0.00/473 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/261M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

Device set to use cuda:0


* Running on local URL:  http://127.0.0.1:7860
It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

* Running on public URL: https://20d58d8b144a6f21d4.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
